In [ ]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
imb_19 = pd.read_csv('../data/imbalance_1h_2019.csv')
imb_19

In [ ]:
# renaming columns
imb_19 = imb_19.rename(columns={'timestamp': 'ds', 'imbalance_price': 'y'})

In [ ]:
# split
def train_test_split(df):
    n = int(len(df)*0.7)
    train, test = df.iloc[:n], df.iloc[n:]

    return train,test

In [ ]:
train_19, test_19 = train_test_split(imb_19)
train_19.shape

In [ ]:
test_19.shape

### Parameters
1) growth
2) changepoints
3) changepoint_range
4) changepoint_prior_scale
5) fourier_order

In [ ]:
# model instantiation
m = Prophet(
    growth="linear", 
    changepoint_prior_scale=200, 
    seasonality_prior_scale=50,
    daily_seasonality=35,
    weekly_seasonality=60,
    yearly_seasonality=40
    )

In [ ]:
m.fit(train_19)

In [ ]:
# periods of timesteps in dataframe or periods of forecast?!
future = m.make_future_dataframe(periods=1, freq='H')
future.shape

In [ ]:
future

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
forecast_19 = m.plot(forecast)

In [ ]:
model_components_19 = m.plot_components(forecast)

In [ ]:
# cv_19_results = cross_validation(Prophet, ) 
cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')

In [ ]:
# rmse = mean_squared_error(y_true, y_pred , squared=False)
# rmse